In [50]:
import numpy as np
import pandas as pd
import datetime
from sklearn.cross_validation import KFold
from sklearn.cross_validation import train_test_split
import time
from sklearn import preprocessing

from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor, GradientBoostingRegressor
from sklearn.grid_search import GridSearchCV
from sklearn.cross_validation import ShuffleSplit
from sklearn.metrics import make_scorer, mean_squared_error
from sklearn.linear_model import Ridge, LassoCV,LassoLarsCV, ElasticNet
from sklearn.kernel_ridge import KernelRidge
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from scipy.stats import skew


In [42]:

train = pd.read_csv("./train.csv") # read train data
test = pd.read_csv("./test.csv") # read test data

In [43]:
def error(real_value, prediction):
    return mean_squared_error(real_value,prediction)**0.5

RMSE = make_scorer(error, greater_is_better=False)

In [44]:
class ensemble(object):
    def __init__(self,n_folds,stacker,base_models):
        self.n_folds=n_folds
        self.stacker=stacker
        self.base_models=base_models
        
    def fit_perdict(self,train,test,ytr):
        X=train.values
        y=ytr.values
        T=test.values
        
        folds=list(KFold(len(y)),n_folds=self.n_folds,shuffle=True,random_state=0)
        s_train=np.zeros((X.shape[0],len(self.base_models)))
        s_test=np.zeros(T.shape[0],len(self.base_models))
        
        for i, ref in enumerate(base_models):
            print("fittig the base model....")
            
            s_test_i=np.zeros(T.shape[0],len(folds))
            for j ,(train_idx,test_idx) in enumerate(folds):
                x_train=x[train_idx]
                y_train=y[train_idx]
                x_holdout=x[test_idx]
                reg.fit(x_train,y_train)
                y_pred=reg.predict(x_holdout[:])
                s_train[test_idx,i]=y_pred
                s_test_i[:,j]=reg.predict(T)[:]
            s_test[:,i]=s_test_i.mean(1)
            
            
        printing("stacking base models....")
        
        param_grid={
            
          'alpha': [1e-3,5e-3,1e-2,5e-2,1e-1,0.2,0.3,0.4,0.5,0.8,1e0,3,5,7,1e1],
            
        }
        
        grid=GridSearchCV(estimator=self.stacker, param_grid=parma_grid, n_jobs=1,cv=5,scoring=RMSE)
        gird.fit(s_train,y)
        
        try:
            print('Param grid:')
            print(param_grid)
            print('Best Params:')
            print(grid.best_params_)
            print('Best CV Score:')
            print(-grid.best_score_)
            print('Best estimator:')
            print(grid.best_estimator_)
            print(message)
        except:
            pass
        
        y_pred = grid.predict(S_test)[:]
        return y_pred, -grid.best_score_
    

In [45]:
base_models=[
    RandomForestRegressor(
    n_jobs=1, random_state=0,
    n_estimators=500, max_features=14
    
    
    ),
    
     RandomForestRegressor(
    n_jobs=1, random_state=0,
    n_estimators=500, max_features=20,
    max_depth=7
    
    ),
    
    
    ExtraTreesRegressor(
    n_jobs=1, random_state=0,
        n_estimators=500, max_features=15
    
    
    
    )
    ,
    
    ExtraTreesRegressor(
    n_jobs=1, random_state=0,
        n_estimators=500, max_features=20
    
    
    
    )
    ,
    
    GradientBoostingRegressor(
    
    random_state=0, 
        n_estimators=500, max_features=10, max_depth=6,
        learning_rate=0.05, subsample=0.8
    
    
    ),
    
    GradientBoostingRegressor(
    
    random_state=0, 
        n_estimators=500, max_features=15, max_depth=6,
        learning_rate=0.05, subsample=0.8
    
    
    )
,
    
    
    LassoCV(alphas=[1,0.1,0.001,0.0005]),
    KNeighborsRegressor(n_neighbors=5),
    KNeighborsRegressor(n_neighbors=10),
    KNeighborsRegressor(n_neighbors=15),
    KNeighborsRegressor(n_neighbors=25),
    LassoLarsCV(),
    ElasticNet(),
    SVR()
    
    
]




In [46]:
def data_preprocess(train, test):
    outlier_idx=[   4,   11,   13,   30,   53,   66,  112,  142,  151,  153,  185,
        199,  218,  224,  231,  238,  261,  271,  313,  318,  328,  377,
        410,  440,  451,  454,  457,  462,  473,  488,  496,  523,  529,
        559,  568,  581,  583,  588,  595,  607,  608,  628,  632,  666,
        681,  688,  691,  692,  714,  738,  747,  769,  774,  803,  825,
        864,  885,  898,  970,  990, 1046, 1065, 1142, 1169, 1181, 1182,
       1211, 1298, 1322, 1324, 1328, 1359, 1423, 1442, 1453]
    train.drop(train.index[outlier_idx], inplace=True)
    
    all_data = pd.concat([train.loc[:,'MSSubClass':'SaleCondition'], test.loc[:,'MSSubClass':'SaleCondition']])
    
    to_delete = ['Alley','FireplaceQu','PoolQC','Fence','MiscFeature']
    all_data.drop(to_delete, axis=1, inplace = True)
    
    #log transform
    numeric_feats = all_data.dtypes[all_data.dtypes != "object"].index
    skewness = all_data[numeric_feats].apply(lambda x : skew(x.dropna()))
    skewed_feats = skewness[skewness>0.75].index
    all_data[skewed_feats] = np.log1p(all_data[skewed_feats])
    
    #encode categorical data
    all_data=pd.get_dummies(all_data)
    
    #fillna
    all_data = all_data.fillna(all_data.mean())
    
    X_train = all_data[:train.shape[0]]
    X_test = all_data[train.shape[0]:]
    y_train = np.log1p(train.SalePrice)
    
    return X_train, X_test, y_train

In [47]:
x_train,x_test,y_train = data_preprocess(train,test)



In [51]:
ensem=ensemble(

        n_folds=5,
    stacker=Ridge(),
    base_models=base_models



)


y_pred, score = ensem.fit_predict(x_train,x_test,y_train)

AttributeError: 'ensemble' object has no attribute 'fit_predict'